# Installing and Importing Libraries

In [1]:
!pip install pytrends
!pip install huggingface_hub

In [12]:
import requests

In [2]:
import pandas as pd
from pytrends.request import TrendReq
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from huggingface_hub import login

# Logging into HubbingFace account and Mounting Google drive

In [3]:
from google.colab import userdata
hf_token = login(userdata.get('Surya-hf-token'))
login(token = hf_token)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
file_path = "/content/drive/MyDrive/Design AI/mock_users.csv"

# Importing Model

In [6]:
# ---- Load a Single LLM for All Agents ----
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.float16, device_map="auto")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [7]:
#Intializing Tokenizers
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token

# Response Generator

In [8]:
# Function to generate responses using LLM
def generate_llm_response(input_text):
    # inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
    # outputs = model.generate(**inputs, max_length=250)


    device = model.device
    inputs = tokenizer(input_text, padding=True, return_tensors="pt")  # Move input to GPU

    outputs = model.generate(
            input_ids=inputs['input_ids'].to(device),
            attention_mask=inputs['attention_mask'].to(device),
            pad_token_id= tokenizer.pad_token_id,
            max_new_tokens = 250,  # Adjust based on your needs
            num_return_sequences = 1,
            temperature = 0.7,  # Control randomness or the diversity
            top_p = 0.9,  # Nucleus sampling
            do_sample = True,
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Agents

## Agent1 - Fetching Demand Trends

In [9]:
# ---- Agent 1: Fetch Google Trends and Use LLM for Analysis ----
class Agent1:
    @staticmethod
    def fetch_seasonal_demand(location):
        """
        Fetches seasonal demand trends using Google Trends (Pytrends).
        Then, the shared LLM interprets the data for insights.
        """
        pytrends = TrendReq()
        search_term = f"{location} travel"

        # Fetch Google search trends for the past 12 months
        pytrends.build_payload([search_term], timeframe="today 12-m")
        trend_data = pytrends.interest_over_time()

        if not trend_data.empty:
            avg_trend = trend_data[search_term].mean()  # Calculate the average interest
            trend_summary = f"Google Trends interest score for {location}: {avg_trend:.2f}"
        else:
            trend_summary = f"No trend data available for {location}"

        # Use shared LLM to interpret trends
        input_text = f"Analyze the following travel trend data: {trend_summary}. Provide insights on demand."
        return generate_llm_response(input_text)

## Agent 2 - Reading user data to Summarize Preferences

In [10]:
# ---- Agent 2: Read User Data and Use LLM to Summarize Preferences ----
class Agent2:
    @staticmethod
    def fetch_user_data(user_id, file_path):
        """
        Reads the user CSV file, extracts booking preferences, and uses the shared LLM to summarize them.
        """
        try:
            df = pd.read_csv(file_path)
            user_row = df[df["user_id"] == user_id]
            if not user_row.empty:
                past_bookings = user_row["past_bookings"].values[0].split("|")
                # preferred_locations = user_row["preferred_locations"].values[0].split("|")
                budget = user_row["budget_preference"].values[0]
                amenities = user_row["amenities_preference"].values[0].split("|")
                travel_purpose = user_row["travel_purpose"].values[0]

                user_summary = (f"User has booked {past_bookings} . "
                                f"Budget preference: {budget}. "
                                f"Desired amenities: {amenities}. "
                                f"Travel purpose: {travel_purpose}.")
            else:
                user_summary = "No historical data available for this user."
        except Exception as e:
            user_summary = f"Error reading user data: {e}"

        # Use shared LLM to summarize user behavior
        input_text = f"Summarize the following user booking behavior: {user_summary}."
        return generate_llm_response(input_text)

## Agent 3 - Generating Reccomendations

In [22]:
# retreiving airbnb listing given a location using public opendata soft api
def get_top_airbnbs(city, limit=10):

      """Fetches the top Airbnb listings for a given city with price and room type."""

      base_url = "https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/air-bnb-listings/records"
      params = {
          "limit": 100,  # Fetch up to 100 results to sort later
          "where": f"city='{city}'"  # Filter by city
      }

      # Send GET request
      response = requests.get(base_url, params=params)

      if response.status_code == 200:
          data = response.json()
          listings = data.get("results", [])

          # Sort listings by number of reviews (most reviewed first)
          sorted_listings = sorted(listings, key=lambda x: x.get("number_of_reviews", 0), reverse=True)

          # Prepare list of formatted responses
          top_airbnbs = [
              f"{i+1}. {listing.get('name', 'N/A')} - {listing.get('neighbourhood', 'N/A')} - "
              f"Reviews: {listing.get('number_of_reviews', 0)} - Price: {listing.get('column_10', 'Price not available')} - "
              f"Room Type: {listing.get('room_type', 'Room type not listed')}"
              for i, listing in enumerate(sorted_listings[:limit])
          ]

          return top_airbnbs
      else:
          print("Error:", response.status_code)
          return []

In [19]:

# ---- Agent 3: Generate Recommendations using the Shared LLM ----
class Agent3:

    @staticmethod
    def generate_recommendations(user_id, location):
        """
        Combines insights from Agent 1 and Agent 2 and generates Airbnb recommendations.
        """
        # Get data from Agent 1 and Agent 2
        seasonal_demand = Agent1.fetch_seasonal_demand(location)
        user_prefs = Agent2.fetch_user_data(user_id,file_path)
        budget_ranges = {"Low": (20, 100), "Medium": (100, 250), "High": (250, 1000)}
        top_airbnbs = get_top_airbnbs(location,limit = 20)
        # Combine data and generate recommendations
        input_text = (
            f"User insights: {user_prefs}. "
            f"Location trends: {seasonal_demand}. "
            f"Top Airbnb listings: {top_airbnbs}. "
            f"Budget ranges: {budget_ranges}"
            f"Based on this, suggest 2-3 ideal Airbnb listings that match the user's budget and amenities preference."
        )

        return generate_llm_response(input_text)

# AI System IN ACTION!

In [20]:
# ---- Example Usage ----
user_id = "user_1"
location = "Sydney"  # Example location
recommendations = Agent3.generate_recommendations(user_id, location)
print(recommendations)


/usr/local/lib/python3.11/dist-packages/pytrends/request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)


User insights: Summarize the following user booking behavior: User has booked ['Budget', 'City Center'] . Budget preference: Low. Desired amenities: ['Balcony', 'Kitchen', 'Ocean View']. Travel purpose: Backpacking..

The user has booked a 'Budget' type room located in the 'City Center' with a low budget preference. The desired amenities for the room are a 'Balcony', 'Kitchen', and 'Ocean View'. The travel purpose is backpacking.. Location trends: Analyze the following travel trend data: Google Trends interest score for Sydney: 74.89. Provide insights on demand.

Insights:

Based on the Google Trends interest score of 74.89 for Sydney, it can be inferred that there is a high level of interest in travel to Sydney. An interest score of 75 or above is considered peak popularity, while a score of 25-50 is considered moderate popularity. 

In terms of demand, this indicates that there are a significant number of people searching for information related to traveling to Sydney. This could be 

In [21]:
print(recommendations.encode().decode("unicode_escape"))

User insights: Summarize the following user booking behavior: User has booked ['Budget', 'City Center'] . Budget preference: Low. Desired amenities: ['Balcony', 'Kitchen', 'Ocean View']. Travel purpose: Backpacking..

The user has booked a 'Budget' type room located in the 'City Center' with a low budget preference. The desired amenities for the room are a 'Balcony', 'Kitchen', and 'Ocean View'. The travel purpose is backpacking.. Location trends: Analyze the following travel trend data: Google Trends interest score for Sydney: 74.89. Provide insights on demand.

Insights:

Based on the Google Trends interest score of 74.89 for Sydney, it can be inferred that there is a high level of interest in travel to Sydney. An interest score of 75 or above is considered peak popularity, while a score of 25-50 is considered moderate popularity. 

In terms of demand, this indicates that there are a significant number of people searching for information related to traveling to Sydney. This could be 

In [23]:
user_id = "user_14"
location = "Paris"  # Example location
recommendations = Agent3.generate_recommendations(user_id, location)
print(recommendations)

/usr/local/lib/python3.11/dist-packages/pytrends/request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)


User insights: Summarize the following user booking behavior: User has booked ['Luxury', 'Boutique Hotel'] . Budget preference: Medium. Desired amenities: ['Balcony', 'Pool', 'Kitchen']. Travel purpose: Backpacking..

The user has booked a luxury or boutique hotel with a medium budget. They prefer a balcony, pool, and kitchen as amenities. Their travel purpose is backpacking.. Location trends: Analyze the following travel trend data: Google Trends interest score for Paris: 70.08. Provide insights on demand.

Insights: Based on the Google Trends interest score for Paris of 70.08, it can be inferred that there is a high level of interest in travel to Paris. This interest score is on a scale of 0 to 100, where 100 represents peak interest and 0 represents no interest at all. A score of 70.08 indicates that Paris is a highly popular travel destination, with a significant number of people searching for information related to travel to Paris. This suggests that there is a high demand for tra

In [24]:
print(recommendations.encode().decode("unicode_escape"))

User insights: Summarize the following user booking behavior: User has booked ['Luxury', 'Boutique Hotel'] . Budget preference: Medium. Desired amenities: ['Balcony', 'Pool', 'Kitchen']. Travel purpose: Backpacking..

The user has booked a luxury or boutique hotel with a medium budget. They prefer a balcony, pool, and kitchen as amenities. Their travel purpose is backpacking.. Location trends: Analyze the following travel trend data: Google Trends interest score for Paris: 70.08. Provide insights on demand.

Insights: Based on the Google Trends interest score for Paris of 70.08, it can be inferred that there is a high level of interest in travel to Paris. This interest score is on a scale of 0 to 100, where 100 represents peak interest and 0 represents no interest at all. A score of 70.08 indicates that Paris is a highly popular travel destination, with a significant number of people searching for information related to travel to Paris. This suggests that there is a high demand for tra